In [ ]:
pip install xgboost

In [ ]:
!pip install tensorflow

In [ ]:
pip install lightgbm

In [ ]:
pip install seaborn

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore') # Игнорируем предупреждения для чистоты вывода

# Определим путь к данным
# Убедитесь, что этот путь корректен относительно расположения вашего ноутбука
try:
    # Попытка использовать путь относительно папки notebooks/
    data_path = '../data/processed/ready_for_training'
    if not os.path.exists(os.path.join(data_path, 'YDEX_processed.csv')):
         # Если не найдено, пробуем путь из корня проекта
         data_path = 'data/processed/ready_for_training'
         if not os.path.exists(os.path.join(data_path, 'YDEX_processed.csv')):
             print("Не удалось автоматически определить путь к данным. Пожалуйста, укажите его вручную в переменной data_path.")
             data_path = input("Введите путь к папке ready_for_training: ") # Запрос пути у пользователя, если автоматически не найден
except Exception as e:
    print(f"Ошибка при определении пути: {e}")
    data_path = 'data/processed/ready_for_training' # Значение по умолчанию

tickers = ['YDEX', 'SBER', 'GAZP', 'DELI']
datasets = {}

print(f"Используемый путь к данным: {data_path}")

# Загрузим данные
for ticker in tickers:
    file_path = os.path.join(data_path, f'{ticker}_processed.csv')
    try:
        if os.path.exists(file_path):
            datasets[ticker] = pd.read_csv(file_path, parse_dates=['DATE'], index_col='DATE')
            # Сортируем данные по дате на всякий случай
            datasets[ticker] = datasets[ticker].sort_index()
            print(f"Загружен {ticker}: {datasets[ticker].shape}")
        else:
            print(f"Файл для {ticker} не найден: {file_path}")
    except Exception as e:
        print(f"Ошибка при загрузке {ticker}: {e}")


# Возьмем один датасет для примера определения колонок
sample_ticker = None
for t in tickers:
    if t in datasets:
        sample_ticker = t
        break

if sample_ticker and sample_ticker in datasets:
    df_sample = datasets[sample_ticker]

    # Определяем целевые колонки
    target_columns = [col for col in df_sample.columns if 'target' in col]
    print(f"\nЦелевые колонки ({len(target_columns)}): {target_columns}")

    # Определяем колонки признаков (все числовые, кроме целей и идентификаторов)
    # Убедимся, что DATE и SECID не попали в признаки (DATE теперь индекс)
    exclude_cols = target_columns + ['SECID']
    feature_columns = [col for col in df_sample.select_dtypes(include=[np.number]).columns if col not in exclude_cols]
    print(f"Колонки признаков ({len(feature_columns)}): {feature_columns[:10]}...{feature_columns[-5:]}") # Печатаем часть для краткости

    # Проверим типы данных и наличие пропусков
    print(f"\nИнформация о датасете (пример {sample_ticker}):")
    df_sample.info()
    # Посчитаем % пропусков
    missing_percentage = (df_sample[feature_columns + target_columns].isnull().sum() / len(df_sample) * 100).sort_values(ascending=False)
    print(f"\nПроцент пропусков (топ-5, пример {sample_ticker}):")
    print(missing_percentage[missing_percentage > 0].head())

    # Удалим строки, где есть пропуски в целевых переменных (без них обучение невозможно)
    for ticker in datasets:
        initial_rows = len(datasets[ticker])
        datasets[ticker] = datasets[ticker].dropna(subset=target_columns)
        removed_rows = initial_rows - len(datasets[ticker])
        if removed_rows > 0:
            print(f"{ticker}: Удалено {removed_rows} строк с пропусками в target.")

else:
    print("Не удалось загрузить ни один из указанных датасетов.")

In [ ]:
from sklearn.model_selection import train_test_split # Хотя нам нужно временное разделение
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error

import xgboost as xgb
import lightgbm as lgb
import catboost as cb
from sklearn.linear_model import LinearRegression
# Импорты для LSTM добавим позже, когда будем его реализовывать

# --- Параметры разделения ---
TRAIN_SIZE = 0.70
VALIDATION_SIZE = 0.15
# TEST_SIZE будет 1.0 - TRAIN_SIZE - VALIDATION_SIZE = 0.15

# --- Словари для хранения данных ---
split_data = {} # Хранение разделенных и обработанных данных {ticker: {target: {'X_train': ..., 'y_train': ..., ...}}}
scalers = {}    # Хранение обученных скейлеров {ticker: scaler}
imputers = {}   # Хранение обученных импьютеров {ticker: imputer}

# --- Цикл по тикерам для разделения и преобработки ---
for ticker, df in datasets.items():
    print(f"\nОбработка {ticker}...")
    if df.empty:
        print(f"Датасет для {ticker} пуст, пропуск.")
        continue

    # Убедимся что feature_columns актуальны для текущего тикера (на случай разной структуры)
    current_target_columns = [col for col in df.columns if 'target' in col]
    current_exclude_cols = current_target_columns + ['SECID']
    current_feature_columns = [col for col in df.select_dtypes(include=[np.number]).columns if col not in current_exclude_cols]

    if not current_feature_columns:
         print(f"Не найдены числовые признаки для {ticker}, пропуск.")
         continue

    X = df[current_feature_columns]
    y = df[current_target_columns] # Пока берем все таргеты

    # --- Временное разделение ---
    n = len(df)
    train_end_idx = int(n * TRAIN_SIZE)
    validation_end_idx = int(n * (TRAIN_SIZE + VALIDATION_SIZE))

    X_train = X.iloc[:train_end_idx]
    X_val = X.iloc[train_end_idx:validation_end_idx]
    X_test = X.iloc[validation_end_idx:]

    y_train = y.iloc[:train_end_idx]
    y_val = y.iloc[train_end_idx:validation_end_idx]
    y_test = y.iloc[validation_end_idx:]

    print(f"  Размеры выборок для {ticker}: Train={len(X_train)}, Validation={len(X_val)}, Test={len(X_test)}")

    if len(X_train) == 0 or len(X_val) == 0 or len(X_test) == 0:
        print(f"  Недостаточно данных для разделения {ticker}, пропуск.")
        continue

    # --- Обработка пропусков (Imputation) ---
    # Заменяем бесконечные значения на NaN перед импутацией
    X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
    X_val.replace([np.inf, -np.inf], np.nan, inplace=True)
    X_test.replace([np.inf, -np.inf], np.nan, inplace=True)

    imputer = SimpleImputer(strategy='median')
    X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
    X_val_imputed = pd.DataFrame(imputer.transform(X_val), columns=X_val.columns, index=X_val.index)
    X_test_imputed = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns, index=X_test.index)
    imputers[ticker] = imputer # Сохраняем импьютер

    # --- Масштабирование (Scaling) ---
    scaler = StandardScaler()
    # Обучаем скейлер только на обучающей выборке (уже без пропусков)
    X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train_imputed), columns=X_train.columns, index=X_train.index)
    X_val_scaled = pd.DataFrame(scaler.transform(X_val_imputed), columns=X_val.columns, index=X_val.index)
    X_test_scaled = pd.DataFrame(scaler.transform(X_test_imputed), columns=X_test.columns, index=X_test.index)
    scalers[ticker] = scaler # Сохраняем скейлер

    # --- Сохранение данных для каждого таргета ---
    split_data[ticker] = {}
    for target_col in current_target_columns:
        # Проверим наличие NaN в y после удаления строк на предыдущем шаге
        if y_train[target_col].isnull().any() or y_val[target_col].isnull().any() or y_test[target_col].isnull().any():
             print(f"  Внимание: Найдены NaN в '{target_col}' для {ticker} ПОСЛЕ первоначальной очистки. Пропуск этого таргета.")
             continue

        split_data[ticker][target_col] = {
            'X_train': X_train_scaled, # Используем масштабированные и импутированные данные для большинства моделей
            'y_train': y_train[target_col],
            'X_val': X_val_scaled,
            'y_val': y_val[target_col],
            'X_test': X_test_scaled,
            'y_test': y_test[target_col],
            # Сохраним и немасштабированные (но с импутацией!) для моделей, которые этого не требуют (деревья)
            'X_train_orig': X_train_imputed,
            'X_val_orig': X_val_imputed,
            'X_test_orig': X_test_imputed,
        }
    print(f"  Данные для {ticker} разделены и предобработаны.")

# --- Определим модели (пока без LSTM) ---
models = {
    "LinearRegression": LinearRegression(),
    "XGBoost": xgb.XGBRegressor(random_state=42, n_estimators=100, early_stopping_rounds=10), # n_estimators и early_stopping - пример
    "LightGBM": lgb.LGBMRegressor(random_state=42, n_estimators=100, learning_rate=0.1), # n_estimators и learning_rate - пример
    "CatBoost": cb.CatBoostRegressor(random_state=42, iterations=100, verbose=0, early_stopping_rounds=10) # iterations и early_stopping - пример
}

# --- Словарь для хранения результатов ---
results = {} # {ticker: {target: {model_name: {'metrics': {...}, 'predictions': ...}}}}

print("\nПодготовка завершена. Данные разделены и предобработаны.")
print(f"Доступные тикеры для обучения: {list(split_data.keys())}")
if split_data:
    # Найдем первый доступный тикер и таргет для примера
    first_ticker = next(iter(split_data), None)
    if first_ticker and split_data[first_ticker]:
        first_target = next(iter(split_data[first_ticker]), None)
        if first_target:
             print(f"Пример структуры split_data[{first_ticker}][{first_target}].keys(): {split_data[first_ticker][first_target].keys()}")
        else:
             print(f"Нет доступных таргетов для тикера {first_ticker}")
    else:
         print("Нет доступных тикеров с данными.")

In [ ]:
import time
from math import sqrt # Для RMSE

def calculate_metrics(y_true, y_pred):
    """Вычисляет набор метрик регрессии."""
    metrics = {}
    metrics['mae'] = mean_absolute_error(y_true, y_pred)
    metrics['mse'] = mean_squared_error(y_true, y_pred)
    metrics['rmse'] = sqrt(metrics['mse'])
    metrics['r2'] = r2_score(y_true, y_pred)
    # MAPE может вызвать ошибку, если y_true содержит нули
    try:
        # Заменяем нули в y_true на очень маленькое число, чтобы избежать деления на ноль
        y_true_safe = np.where(y_true == 0, 1e-9, y_true)
        metrics['mape'] = mean_absolute_percentage_error(y_true_safe, y_pred)
    except Exception as e:
        print(f"  Предупреждение: Не удалось рассчитать MAPE: {e}")
        metrics['mape'] = np.nan
    return metrics

# --- Основной цикл обучения и оценки ---
training_start_time = time.time()

for ticker in split_data.keys():
    print(f"\n===== Обучение для тикера: {ticker} =====")
    if ticker not in results:
        results[ticker] = {}

    for target_col in split_data[ticker].keys():
        print(f"  --- Таргет: {target_col} ---")
        if target_col not in results[ticker]:
            results[ticker][target_col] = {}

        # Извлекаем нужные данные для текущего таргета
        data = split_data[ticker][target_col]
        y_train = data['y_train']
        y_val = data['y_val']
        y_test = data['y_test']

        # Проверка на случай, если y_val или y_test пустые (хотя не должно быть после проверок)
        if y_val.empty or y_test.empty:
            print(f"    Пропуск таргета {target_col} для {ticker} из-за пустых y_val или y_test.")
            continue

        for model_name, model in models.items():
            print(f"    Модель: {model_name}")
            start_model_time = time.time()

            # Выбираем данные (масштабированные или оригинальные)
            if model_name == "LinearRegression":
                X_train_fit = data['X_train']
                X_val_fit = data['X_val']
                X_test_predict = data['X_test']
            else: # Для деревьев используем оригинальные (но импутированные)
                X_train_fit = data['X_train_orig']
                X_val_fit = data['X_val_orig']
                X_test_predict = data['X_test_orig']

            try:
                # Обучение модели
                if model_name == "XGBoost":
                    model.fit(X_train_fit, y_train,
                              eval_set=[(X_val_fit, y_val)],
                              verbose=False)
                elif model_name == "LightGBM":
                    # LightGBM требует имя параметра early_stopping_rounds в fit
                    callbacks = [lgb.early_stopping(stopping_rounds=10, verbose=False)]
                    model.fit(X_train_fit, y_train,
                              eval_set=[(X_val_fit, y_val)],
                              callbacks=callbacks)
                elif model_name == "CatBoost":
                     # CatBoost имеет параметр early_stopping_rounds в конструкторе
                     # но также может принимать eval_set в fit
                     model.fit(X_train_fit, y_train,
                               eval_set=[(X_val_fit, y_val)],
                               verbose=0) # verbose=0 уже был в конструкторе, дублируем для ясности
                else: # LinearRegression
                    model.fit(X_train_fit, y_train)

                # Предсказание
                y_pred = model.predict(X_test_predict)

                # Расчет метрик
                metrics = calculate_metrics(y_test, y_pred)
                results[ticker][target_col][model_name] = {
                    'metrics': metrics,
                    'predictions': pd.Series(y_pred, index=y_test.index) # Сохраняем предсказания с индексами
                }
                print(f"      MAE: {metrics['mae']:.4f}, RMSE: {metrics['rmse']:.4f}, R2: {metrics['r2']:.4f}, MAPE: {metrics['mape']:.4f}")

            except Exception as e:
                print(f"    Ошибка при обучении/оценке модели {model_name} для {ticker}-{target_col}: {e}")
                results[ticker][target_col][model_name] = {'metrics': {}, 'predictions': None, 'error': str(e)}

            end_model_time = time.time()
            print(f"      Время: {end_model_time - start_model_time:.2f} сек.")


training_end_time = time.time()
print(f"\n===== Обучение завершено! Общее время: {training_end_time - training_start_time:.2f} сек. =====")

# Выведем пример структуры результатов
if results:
    first_ticker = next(iter(results))
    if results[first_ticker]:
        first_target = next(iter(results[first_ticker]))
        if results[first_ticker][first_target]:
            first_model = next(iter(results[first_ticker][first_target]))
            print(f"\nПример структуры results[{first_ticker}][{first_target}][{first_model}]:")
            print(results[first_ticker][first_target][first_model]['metrics'])


In [ ]:
# --- Сбор метрик в DataFrame ---
all_metrics_data = []

for ticker, targets_data in results.items():
    for target_col, models_data in targets_data.items():
        for model_name, model_results in models_data.items():
            if 'metrics' in model_results and model_results['metrics']: # Проверяем, что метрики существуют
                metrics = model_results['metrics']
                metrics['ticker'] = ticker
                metrics['target'] = target_col
                metrics['model'] = model_name
                all_metrics_data.append(metrics)
            elif 'error' in model_results: # Записываем информацию об ошибке, если она была
                 all_metrics_data.append({
                     'ticker': ticker,
                     'target': target_col,
                     'model': model_name,
                     'mae': np.nan, 'mse': np.nan, 'rmse': np.nan, 'r2': np.nan, 'mape': np.nan,
                     'error': model_results['error']
                 })


metrics_df = pd.DataFrame(all_metrics_data)

# Упорядочим колонки для лучшей читаемости
metric_cols = ['mae', 'rmse', 'r2', 'mape'] # Основные метрики
id_cols = ['ticker', 'target', 'model']
other_cols = [col for col in metrics_df.columns if col not in metric_cols + id_cols] # mse, error и т.д.
metrics_df = metrics_df[id_cols + metric_cols + other_cols]

# --- Отображение метрик ---
print("Сводная таблица метрик по всем моделям, тикерам и таргетам:")

# Отсортируем для наглядности
metrics_df_sorted = metrics_df.sort_values(by=['ticker', 'target', 'rmse']) # Сортируем по RMSE как основной метрике

# Используем display для красивого вывода DataFrame в Jupyter
from IPython.display import display
display(metrics_df_sorted)

# --- Найдем лучшие модели по RMSE для каждого тикера и таргета ---
best_models = metrics_df_sorted.loc[metrics_df_sorted.groupby(['ticker', 'target'])['rmse'].idxmin()]

print("\nЛучшие модели по RMSE для каждого тикера и таргета:")
display(best_models[['ticker', 'target', 'model', 'rmse', 'mae', 'r2', 'mape']])

In [ ]:
# --- Визуализация: Сравнение моделей ---

# Установим стиль seaborn для графиков
sns.set(style="whitegrid")

# Перебираем тикеры
for ticker in metrics_df['ticker'].unique():
    # Фильтруем данные для текущего тикера
    ticker_metrics = metrics_df[metrics_df['ticker'] == ticker]

    # Создаем сетку графиков: одна строка, количество столбцов = количество таргетов
    n_targets = len(ticker_metrics['target'].unique())
    fig, axes = plt.subplots(1, n_targets, figsize=(5 * n_targets, 5), sharey=True) # Share Y-axis for better comparison if scales are similar
    fig.suptitle(f'Сравнение моделей по RMSE для тикера: {ticker}', fontsize=16, y=1.02)

    # Перебираем таргеты и строим график для каждого
    for i, target_col in enumerate(sorted(ticker_metrics['target'].unique())): # Сортируем таргеты для порядка
        ax = axes[i] if n_targets > 1 else axes # Handle single target case
        target_data = ticker_metrics[ticker_metrics['target'] == target_col]

        # Строим столбчатую диаграмму RMSE
        sns.barplot(x='model', y='rmse', data=target_data, ax=ax, palette='viridis')
        ax.set_title(f'Таргет: {target_col}')
        ax.set_xlabel('Модель')
        ax.set_ylabel('RMSE' if i == 0 else '') # Подпись оси Y только для первого графика
        ax.tick_params(axis='x', rotation=45) # Поворачиваем подписи моделей для читаемости

    plt.tight_layout(rect=[0, 0, 1, 0.98]) # Adjust layout to prevent title overlap
    plt.show()

In [ ]:
# --- Визуализация: Сравнение таргетов ---

# Перебираем тикеры
for ticker in metrics_df['ticker'].unique():
    print(f"--- Графики для тикера: {ticker} ---")
    ticker_metrics = metrics_df[metrics_df['ticker'] == ticker].copy() # Берем данные тикера

    # Преобразуем таргет в число для сортировки (1d -> 1, 3d -> 3, ...)
    ticker_metrics['target_days'] = ticker_metrics['target'].str.extract('(\d+)').astype(int)
    ticker_metrics.sort_values(by='target_days', inplace=True)

    # Создаем сетку графиков: одна строка, количество столбцов = количество моделей
    n_models = len(ticker_metrics['model'].unique())
    fig, axes = plt.subplots(1, n_models, figsize=(5 * n_models, 5), sharey=False) # Оси Y могут сильно отличаться
    fig.suptitle(f'Сравнение RMSE по таргетам для тикера: {ticker}', fontsize=16, y=1.02)

    # Перебираем модели и строим график для каждой
    for i, model_name in enumerate(sorted(ticker_metrics['model'].unique())):
        ax = axes[i] if n_models > 1 else axes
        model_data = ticker_metrics[ticker_metrics['model'] == model_name]

        # Строим столбчатую диаграмму RMSE
        sns.barplot(x='target', y='rmse', data=model_data, ax=ax, palette='magma', order=sorted(ticker_metrics['target'].unique(), key=lambda x: int(x.split('d')[0].split('_')[-1])))
        ax.set_title(f'Модель: {model_name}')
        ax.set_xlabel('Таргет')
        ax.set_ylabel('RMSE' if i == 0 else '') # Подпись оси Y только для первого графика
        ax.tick_params(axis='x', rotation=45)

    plt.tight_layout(rect=[0, 0, 1, 0.98])
    plt.show()

In [ ]:
# --- Визуализация: Факт vs Прогноз ---

# Выберем тикер и таргет для примера
example_ticker = 'SBER'
example_target = 'target_7d'

if example_ticker in results and example_target in results[example_ticker]:
    # Найдем лучшую модель для этого примера
    best_model_name = best_models[(best_models['ticker'] == example_ticker) & (best_models['target'] == example_target)]['model'].iloc[0]
    print(f"\nГрафик Факт vs Прогноз для {example_ticker}, таргет {example_target}, модель: {best_model_name}")

    # Получаем реальные данные и предсказания
    y_test_actual = split_data[example_ticker][example_target]['y_test']
    predictions = results[example_ticker][example_target][best_model_name]['predictions']

    if predictions is not None and not y_test_actual.empty:
        plt.figure(figsize=(15, 6))
        plt.plot(y_test_actual.index, y_test_actual, label='Реальные значения (Test)', color='blue', marker='.')
        plt.plot(predictions.index, predictions, label='Предсказания модели (Test)', color='red', linestyle='--', marker='x')
        plt.title(f'{example_ticker} - {example_target}: Реальные vs Предсказанные ({best_model_name})')
        plt.xlabel('Дата')
        plt.ylabel('Значение таргета')
        plt.legend()
        plt.grid(True)
        plt.show()
    else:
        print("Не удалось построить график: отсутствуют предсказания или реальные данные.")
else:
    print(f"Не найдены результаты для {example_ticker} / {example_target} для построения графика Факт vs Прогноз.")

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

def create_sequences(X_data, y_data, sequence_length):
    """Преобразует временные ряды в последовательности для LSTM."""
    X_seq, y_seq = [], []
    for i in range(len(X_data) - sequence_length):
        X_seq.append(X_data.iloc[i:(i + sequence_length)].values)
        y_seq.append(y_data.iloc[i + sequence_length])
    return np.array(X_seq), np.array(y_seq)

def build_lstm_model(sequence_length, n_features):
    """Строит простую LSTM модель."""
    model = Sequential([
        LSTM(50, activation='relu', input_shape=(sequence_length, n_features), return_sequences=True), # Первый слой LSTM
        Dropout(0.2),
        LSTM(50, activation='relu'), # Второй слой LSTM
        Dropout(0.2),
        Dense(25, activation='relu'), # Полносвязный слой
        Dense(1) # Выходной слой (1 нейрон для регрессии)
    ])
    # Используем Adam с пониженной скоростью обучения по умолчанию
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='mse') # Компилируем с MSE loss
    return model

print("Импорты и функции для LSTM добавлены.")

In [ ]:
# --- Параметры LSTM ---
SEQUENCE_LENGTH = 20 # Длина последовательности для входа LSTM
EPOCHS = 50          # Максимальное количество эпох
BATCH_SIZE = 32      # Размер батча

# --- Цикл обучения LSTM ---
lstm_start_time = time.time()

for ticker in split_data.keys():
    print(f"\n===== LSTM Обучение для тикера: {ticker} =====")
    if ticker not in results:
        results[ticker] = {} # На всякий случай, если вдруг не создалось

    for target_col in split_data[ticker].keys():
        print(f"  --- LSTM Таргет: {target_col} ---")
        if target_col not in results[ticker]:
            results[ticker][target_col] = {}

        model_name = "LSTM"
        start_model_time = time.time()

        try:
            # Извлекаем масштабированные данные
            data = split_data[ticker][target_col]
            X_train_scaled = data['X_train']
            y_train = data['y_train']
            X_val_scaled = data['X_val']
            y_val = data['y_val']
            X_test_scaled = data['X_test']
            y_test = data['y_test']

            # Создаем последовательности
            X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train, SEQUENCE_LENGTH)
            X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val, SEQUENCE_LENGTH)
            X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test, SEQUENCE_LENGTH)

            # Проверяем, достаточно ли данных после создания последовательностей
            if len(X_train_seq) == 0 or len(X_val_seq) == 0 or len(X_test_seq) == 0:
                print(f"    Недостаточно данных для создания последовательностей (seq_len={SEQUENCE_LENGTH}). Пропуск LSTM для {ticker}-{target_col}.")
                results[ticker][target_col][model_name] = {'metrics': {}, 'predictions': None, 'error': 'Insufficient data for sequences'}
                continue

            n_features = X_train_seq.shape[2] # Количество признаков

            # Строим модель
            lstm_model = build_lstm_model(SEQUENCE_LENGTH, n_features)
            # print(lstm_model.summary()) # Можно раскомментировать для просмотра структуры модели

            # Коллбэки для обучения
            early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=0)
            reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6, verbose=0)

            # Обучение модели
            history = lstm_model.fit(X_train_seq, y_train_seq,
                                     epochs=EPOCHS,
                                     batch_size=BATCH_SIZE,
                                     validation_data=(X_val_seq, y_val_seq),
                                     callbacks=[early_stopping, reduce_lr],
                                     verbose=0) # verbose=0 чтобы не выводить лог каждой эпохи

            # Предсказание
            y_pred_seq = lstm_model.predict(X_test_seq, verbose=0).flatten() # predict возвращает (n_samples, 1), flatten нужен

            # Расчет метрик (используем y_test_seq, т.к. предсказания сделаны на X_test_seq)
            metrics = calculate_metrics(y_test_seq, y_pred_seq)

            # Сохраняем результаты
            # Для сопоставления предсказаний с датами, берем индекс из y_test, начиная с SEQUENCE_LENGTH-го элемента
            prediction_index = y_test.index[SEQUENCE_LENGTH:]
            results[ticker][target_col][model_name] = {
                'metrics': metrics,
                'predictions': pd.Series(y_pred_seq, index=prediction_index)
            }
            print(f"      LSTM MAE: {metrics['mae']:.4f}, RMSE: {metrics['rmse']:.4f}, R2: {metrics['r2']:.4f}, MAPE: {metrics['mape']:.4f}")

        except Exception as e:
            print(f"    Ошибка при обучении/оценке LSTM для {ticker}-{target_col}: {e}")
            results[ticker][target_col][model_name] = {'metrics': {}, 'predictions': None, 'error': str(e)}

        end_model_time = time.time()
        print(f"      LSTM Время: {end_model_time - start_model_time:.2f} сек.")

lstm_end_time = time.time()
print(f"\n===== LSTM Обучение завершено! Общее время: {lstm_end_time - lstm_start_time:.2f} сек. =====")

# Обновим DataFrame с метриками, включив LSTM
all_metrics_data = []
for ticker, targets_data in results.items():
    for target_col, models_data in targets_data.items():
        for model_name, model_results in models_data.items():
            if 'metrics' in model_results and model_results['metrics']: # Проверяем, что метрики существуют
                metrics = model_results['metrics'].copy() # Копируем, чтобы не изменить исходный словарь
                metrics['ticker'] = ticker
                metrics['target'] = target_col
                metrics['model'] = model_name
                all_metrics_data.append(metrics)
            elif 'error' in model_results:
                 all_metrics_data.append({
                     'ticker': ticker, 'target': target_col, 'model': model_name,
                     'mae': np.nan, 'mse': np.nan, 'rmse': np.nan, 'r2': np.nan, 'mape': np.nan,
                     'error': model_results['error']
                 })

metrics_df_updated = pd.DataFrame(all_metrics_data)
# Упорядочим колонки
metric_cols = ['mae', 'rmse', 'r2', 'mape']
id_cols = ['ticker', 'target', 'model']
other_cols = [col for col in metrics_df_updated.columns if col not in metric_cols + id_cols]
metrics_df_updated = metrics_df_updated[id_cols + metric_cols + other_cols]

# --- Отображение обновленных метрик ---
print("\nОбновленная сводная таблица метрик (включая LSTM):")
metrics_df_sorted_updated = metrics_df_updated.sort_values(by=['ticker', 'target', 'rmse'])
display(metrics_df_sorted_updated)

# --- Обновленные лучшие модели ---
best_models_updated = metrics_df_sorted_updated.loc[metrics_df_sorted_updated.groupby(['ticker', 'target'])['rmse'].idxmin()]
print("\nОбновленные лучшие модели по RMSE для каждого тикера и таргета:")
display(best_models_updated[['ticker', 'target', 'model', 'rmse', 'mae', 'r2', 'mape']])


In [ ]:
# --- Визуализация: Сравнение моделей (включая LSTM) ---

# Перебираем тикеры
for ticker in metrics_df_updated['ticker'].unique():
    # Фильтруем данные для текущего тикера
    ticker_metrics = metrics_df_updated[metrics_df_updated['ticker'] == ticker]

    # Создаем сетку графиков: одна строка, количество столбцов = количество таргетов
    targets = sorted(ticker_metrics['target'].unique(), key=lambda x: int(x.split('d')[0].split('_')[-1])) # Сортируем таргеты
    n_targets = len(targets)
    if n_targets == 0: continue # Пропускаем, если нет таргетов для тикера

    fig, axes = plt.subplots(1, n_targets, figsize=(5 * n_targets, 5), sharey=True) # Share Y-axis
    if n_targets == 1: # Если только один таргет, axes не будет массивом
        axes = [axes]
    fig.suptitle(f'Сравнение моделей по RMSE для тикера: {ticker} (с LSTM)', fontsize=16, y=1.02)

    # Перебираем таргеты и строим график для каждого
    for i, target_col in enumerate(targets):
        ax = axes[i]
        target_data = ticker_metrics[ticker_metrics['target'] == target_col]

        # Строим столбчатую диаграмму RMSE
        sns.barplot(x='model', y='rmse', data=target_data, ax=ax, palette='viridis', order=sorted(target_data['model'].unique())) # Сортируем модели для порядка
        ax.set_title(f'Таргет: {target_col}')
        ax.set_xlabel('Модель')
        ax.set_ylabel('RMSE' if i == 0 else '') # Подпись оси Y только для первого графика
        ax.tick_params(axis='x', rotation=45) # Поворачиваем подписи моделей для читаемости

    plt.tight_layout(rect=[0, 0, 1, 0.98]) # Adjust layout
    plt.show()

In [ ]:
# --- Визуализация: Сравнение таргетов (включая LSTM) ---

# Перебираем тикеры
for ticker in metrics_df_updated['ticker'].unique():
    print(f"--- Графики для тикера: {ticker} (с LSTM) ---")
    ticker_metrics = metrics_df_updated[metrics_df_updated['ticker'] == ticker].copy() # Берем данные тикера

    if ticker_metrics.empty: continue

    # Преобразуем таргет в число для сортировки (1d -> 1, 3d -> 3, ...)
    ticker_metrics['target_days'] = ticker_metrics['target'].str.extract('(\d+)').astype(int)
    ticker_metrics.sort_values(by='target_days', inplace=True)
    sorted_targets = sorted(ticker_metrics['target'].unique(), key=lambda x: int(x.split('d')[0].split('_')[-1]))

    # Создаем сетку графиков: одна строка, количество столбцов = количество моделей
    models_to_plot = sorted(ticker_metrics['model'].unique())
    n_models = len(models_to_plot)
    if n_models == 0: continue

    fig, axes = plt.subplots(1, n_models, figsize=(5 * n_models, 5), sharey=False) # Оси Y могут сильно отличаться
    if n_models == 1:
        axes = [axes]
    fig.suptitle(f'Сравнение RMSE по таргетам для тикера: {ticker} (с LSTM)', fontsize=16, y=1.02)

    # Перебираем модели и строим график для каждой
    for i, model_name in enumerate(models_to_plot):
        ax = axes[i]
        model_data = ticker_metrics[ticker_metrics['model'] == model_name]

        # Строим столбчатую диаграмму RMSE
        sns.barplot(x='target', y='rmse', data=model_data, ax=ax, palette='magma', order=sorted_targets)
        ax.set_title(f'Модель: {model_name}')
        ax.set_xlabel('Таргет')
        ax.set_ylabel('RMSE' if i == 0 else '') # Подпись оси Y только для первого графика
        ax.tick_params(axis='x', rotation=45)

    plt.tight_layout(rect=[0, 0, 1, 0.98])
    plt.show()

In [ ]:
def calculate_directional_accuracy(y_true, y_pred):
    """Расчет точности предсказания знака."""
    # Сравниваем знаки. np.sign возвращает 1, -1 или 0.
    # Считаем совпадение, если знаки одинаковые (включая случай, когда оба 0)
    correct_direction = (np.sign(y_true) == np.sign(y_pred))
    if len(correct_direction) == 0:
        return np.nan # Невозможно посчитать для пустых данных
    return np.mean(correct_direction) * 100 # Возвращаем в процентах

# --- Пересчитаем метрики, добавив directional accuracy ---
# Используем уже существующий словарь results

for ticker in results.keys():
    if not isinstance(results[ticker], dict): continue # Пропуск, если структура некорректна
    for target_col in results[ticker].keys():
         if not isinstance(results[ticker][target_col], dict): continue
         for model_name in results[ticker][target_col].keys():
            if not isinstance(results[ticker][target_col][model_name], dict): continue

            model_results = results[ticker][target_col][model_name]
            if 'predictions' in model_results and model_results['predictions'] is not None:
                y_pred = model_results['predictions']
                # Получаем y_true, соответствующий индексам y_pred
                y_true = split_data[ticker][target_col]['y_test'][y_pred.index]

                if not y_true.empty and not y_pred.empty:
                    dir_acc = calculate_directional_accuracy(y_true.values, y_pred.values)
                    # Добавляем новую метрику в существующий словарь
                    results[ticker][target_col][model_name]['metrics']['dir_acc'] = dir_acc
                else:
                    results[ticker][target_col][model_name]['metrics']['dir_acc'] = np.nan
            elif 'metrics' in model_results: # Если были ошибки или нет предсказаний
                 results[ticker][target_col][model_name]['metrics']['dir_acc'] = np.nan


# --- Снова соберем DataFrame с обновленными метриками ---
all_metrics_data_updated = []
for ticker, targets_data in results.items():
    for target_col, models_data in targets_data.items():
        for model_name, model_results in models_data.items():
            if 'metrics' in model_results and model_results['metrics']:
                metrics = model_results['metrics'].copy()
                metrics['ticker'] = ticker
                metrics['target'] = target_col
                metrics['model'] = model_name
                all_metrics_data_updated.append(metrics)
            # Нет необходимости добавлять записи об ошибках снова, т.к. dir_acc будет NaN

metrics_df_final = pd.DataFrame(all_metrics_data_updated)

# Упорядочим колонки, добавив dir_acc
metric_cols = ['mae', 'rmse', 'r2', 'mape', 'dir_acc'] # Добавили dir_acc
id_cols = ['ticker', 'target', 'model']
other_cols = [col for col in metrics_df_final.columns if col not in metric_cols + id_cols]
metrics_df_final = metrics_df_final[id_cols + metric_cols + other_cols]

# --- Отображение обновленных метрик ---
print("Финальная таблица метрик (с точностью угадывания знака, %):")
metrics_df_final_sorted = metrics_df_final.sort_values(by=['ticker', 'target', 'dir_acc'], ascending=[True, True, False]) # Сортируем по dir_acc по убыванию
display(metrics_df_final_sorted)

# --- Лучшие модели по точности угадывания знака ---
best_dir_acc_models = metrics_df_final_sorted.loc[metrics_df_final_sorted.groupby(['ticker', 'target'])['dir_acc'].idxmax()]
print("\nЛучшие модели по точности угадывания знака (%) для каждого тикера и таргета:")
display(best_dir_acc_models[['ticker', 'target', 'model', 'dir_acc', 'r2', 'rmse']])

In [ ]:
# --- Параметры ---
COMBINED_DATA_FILENAME = 'combined_data.csv' # Уточните, если имя другое
combined_file_path = os.path.join(data_path, COMBINED_DATA_FILENAME)

# --- Загрузка объединенного датасета ---
combined_df = None
if os.path.exists(combined_file_path):
    try:
        combined_df = pd.read_csv(combined_file_path, parse_dates=['DATE'], index_col='DATE')
        combined_df = combined_df.sort_index() # Сортируем по дате
        print(f"Загружен объединенный датасет: {combined_df.shape}")
        print(f"Уникальные тикеры в датасете: {combined_df['SECID'].unique()}")
    except Exception as e:
        print(f"Ошибка при загрузке объединенного датасета '{combined_file_path}': {e}")
else:
    print(f"Объединенный файл не найден: {combined_file_path}")
    print("Пожалуйста, убедитесь, что файл существует и имя указано верно.")

# --- Подготовка признаков и целей (если датасет загружен) ---
if combined_df is not None:
    # Целевые колонки
    combined_target_columns = [col for col in combined_df.columns if 'target' in col]
    print(f"\nЦелевые колонки: {combined_target_columns}")

    # Колонки признаков (все числовые + SECID, кроме целей)
    combined_exclude_cols = combined_target_columns
    # Сначала все числовые
    combined_feature_columns_numeric = [col for col in combined_df.select_dtypes(include=[np.number]).columns if col not in combined_exclude_cols]
    # Добавляем SECID
    combined_feature_columns = combined_feature_columns_numeric + ['SECID']
    categorical_features = ['SECID'] # Список категориальных признаков

    print(f"Колонки признаков ({len(combined_feature_columns)}): {combined_feature_columns[:5]}...{combined_feature_columns[-5:]}")
    print(f"Категориальные признаки: {categorical_features}")

    # Преобразуем категориальный признак для LightGBM
    combined_df['SECID'] = combined_df['SECID'].astype('category')

    # Проверим наличие пропусков в таргетах и удалим строки при необходимости
    initial_rows_combined = len(combined_df)
    combined_df.dropna(subset=combined_target_columns, inplace=True)
    removed_rows_combined = initial_rows_combined - len(combined_df)
    if removed_rows_combined > 0:
        print(f"\nУдалено {removed_rows_combined} строк с пропусками в target из объединенного датасета.")

    # Разделение данных
    X_combined = combined_df[combined_feature_columns]
    y_combined = combined_df[combined_target_columns]

    n_combined = len(combined_df)
    train_end_idx_comb = int(n_combined * TRAIN_SIZE)
    validation_end_idx_comb = int(n_combined * (TRAIN_SIZE + VALIDATION_SIZE))

    X_train_comb = X_combined.iloc[:train_end_idx_comb]
    X_val_comb = X_combined.iloc[train_end_idx_comb:validation_end_idx_comb]
    X_test_comb = X_combined.iloc[validation_end_idx_comb:]

    y_train_comb = y_combined.iloc[:train_end_idx_comb]
    y_val_comb = y_combined.iloc[train_end_idx_comb:validation_end_idx_comb]
    y_test_comb = y_combined.iloc[validation_end_idx_comb:]

    print(f"\nРазмеры выборок для объединенного датасета: Train={len(X_train_comb)}, Validation={len(X_val_comb)}, Test={len(X_test_comb)}")

    if len(X_train_comb) == 0 or len(X_val_comb) == 0 or len(X_test_comb) == 0:
        print(f"  Недостаточно данных для разделения объединенного датасета.")
        combined_df = None # Сбрасываем df, чтобы не продолжать

else:
    print("Обучение на объединенном датасете невозможно без загрузки данных.")

In [ ]:
# --- Модели для объединенного датасета ---
models_combined = {
    # LightGBM должен сам обработать тип 'category' для SECID
    "LightGBM": lgb.LGBMRegressor(random_state=42, n_estimators=100, learning_rate=0.1),
    # Для CatBoost явно указываем категориальный признак
    "CatBoost": cb.CatBoostRegressor(random_state=42, iterations=100, verbose=0, early_stopping_rounds=10,
                                     cat_features=categorical_features) # Передаем список кат. признаков
}

# --- Словарь для хранения результатов объединенного датасета ---
results_combined = {} # {target: {model_name: {'metrics': {...}, 'predictions': ...}}}

# --- Цикл обучения на объединенном датасете ---
if combined_df is not None: # Продолжаем, только если данные были успешно загружены и разделены
    combined_training_start_time = time.time()
    print("\n===== Обучение на ОБЪЕДИНЕННОМ датасете =====")

    for target_col in combined_target_columns:
        print(f"  --- Таргет: {target_col} ---")
        results_combined[target_col] = {}

        # Извлекаем y для текущего таргета
        y_train_target = y_train_comb[target_col]
        y_val_target = y_val_comb[target_col]
        y_test_target = y_test_comb[target_col]

        # Проверка на пустые y
        if y_val_target.empty or y_test_target.empty:
             print(f"    Пропуск таргета {target_col} из-за пустых y_val или y_test.")
             continue

        for model_name, model in models_combined.items():
            print(f"    Модель: {model_name}")
            start_model_time = time.time()

            # Используем X_train_comb, X_val_comb, X_test_comb
            # Масштабирование не применялось, пропуски не заполнялись (модели должны справиться)
            X_train_fit = X_train_comb
            X_val_fit = X_val_comb
            X_test_predict = X_test_comb

            try:
                # Обучение модели
                if model_name == "LightGBM":
                    callbacks = [lgb.early_stopping(stopping_rounds=10, verbose=False)]
                    # Убедимся, что категориальный признак имеет правильный тип
                    X_train_fit_lgb = X_train_fit.copy()
                    X_val_fit_lgb = X_val_fit.copy()
                    X_test_predict_lgb = X_test_predict.copy()
                    for col in categorical_features:
                         if col in X_train_fit_lgb.columns:
                              X_train_fit_lgb[col] = X_train_fit_lgb[col].astype('category')
                              X_val_fit_lgb[col] = X_val_fit_lgb[col].astype('category')
                              X_test_predict_lgb[col] = X_test_predict_lgb[col].astype('category')

                    model.fit(X_train_fit_lgb, y_train_target,
                              eval_set=[(X_val_fit_lgb, y_val_target)],
                              callbacks=callbacks)
                    # Предсказание делаем на данных с правильным типом категории
                    y_pred = model.predict(X_test_predict_lgb)

                elif model_name == "CatBoost":
                     # cat_features уже указан в конструкторе
                     model.fit(X_train_fit, y_train_target,
                               eval_set=[(X_val_fit, y_val_target)],
                               verbose=0)
                     y_pred = model.predict(X_test_predict)

                # Расчет метрик
                metrics = calculate_metrics(y_test_target, y_pred)
                # Добавляем точность угадывания знака
                metrics['dir_acc'] = calculate_directional_accuracy(y_test_target.values, y_pred)

                results_combined[target_col][model_name] = {
                    'metrics': metrics,
                    'predictions': pd.Series(y_pred, index=y_test_target.index)
                }
                print(f"      MAE: {metrics['mae']:.4f}, RMSE: {metrics['rmse']:.4f}, R2: {metrics['r2']:.4f}, MAPE: {metrics['mape']:.4f}, DirAcc: {metrics['dir_acc']:.2f}%")

            except Exception as e:
                print(f"    Ошибка при обучении/оценке модели {model_name} для таргета {target_col}: {e}")
                results_combined[target_col][model_name] = {'metrics': {}, 'predictions': None, 'error': str(e)}

            end_model_time = time.time()
            print(f"      Время: {end_model_time - start_model_time:.2f} сек.")

    combined_training_end_time = time.time()
    print(f"\n===== Обучение на объединенном датасете завершено! Общее время: {combined_training_end_time - combined_training_start_time:.2f} сек. =====")

    # --- Сбор метрик в DataFrame ---
    combined_metrics_data = []
    for target_col, models_data in results_combined.items():
        for model_name, model_results in models_data.items():
            if 'metrics' in model_results and model_results['metrics']:
                metrics = model_results['metrics'].copy()
                metrics['target'] = target_col
                metrics['model'] = model_name
                combined_metrics_data.append(metrics)

    metrics_df_combined = pd.DataFrame(combined_metrics_data)

    # Упорядочим колонки
    metric_cols = ['mae', 'rmse', 'r2', 'mape', 'dir_acc']
    id_cols = ['target', 'model']
    other_cols = [col for col in metrics_df_combined.columns if col not in metric_cols + id_cols]
    metrics_df_combined = metrics_df_combined[id_cols + metric_cols + other_cols]

    # --- Отображение метрик ---
    print("\nСводная таблица метрик для объединенного датасета:")
    metrics_df_combined_sorted = metrics_df_combined.sort_values(by=['target', 'dir_acc'], ascending=[True, False])
    display(metrics_df_combined_sorted)

    # --- Лучшие модели по точности угадывания знака ---
    best_dir_acc_models_combined = metrics_df_combined_sorted.loc[metrics_df_combined_sorted.groupby(['target'])['dir_acc'].idxmax()]
    print("\nЛучшие модели по точности угадывания знака (%) для каждого таргета (объединенный датасет):")
    display(best_dir_acc_models_combined[['target', 'model', 'dir_acc', 'r2', 'rmse']])

else:
    print("\nОбучение на объединенном датасете не выполнено из-за отсутствия данных.")


In [ ]:
# --- Подготовка данных для сравнения ---

# Метрики из индивидуального обучения (только LGBM и CatBoost)
metrics_individual = metrics_df_final[metrics_df_final['model'].isin(['LightGBM', 'CatBoost'])].copy()

# Усредняем метрики по тикерам для каждого таргета и модели
metrics_individual_avg = metrics_individual.groupby(['target', 'model'])[['rmse', 'dir_acc']].mean().reset_index()
metrics_individual_avg['experiment'] = 'Individual (Avg)'

# Метрики из объединенного обучения
metrics_combined_comp = metrics_df_combined[['target', 'model', 'rmse', 'dir_acc']].copy()
metrics_combined_comp['experiment'] = 'Combined'

# Объединяем
comparison_df = pd.concat([metrics_individual_avg, metrics_combined_comp], ignore_index=True)

# Сортируем для наглядности
comparison_df['target_days'] = comparison_df['target'].str.extract('(\d+)').astype(int)
comparison_df.sort_values(by=['target_days', 'model', 'experiment'], inplace=True)

print("Данные для сравнения:")
display(comparison_df[['experiment', 'target', 'model', 'rmse', 'dir_acc']])

In [ ]:
# --- Визуализация: Сравнение RMSE ---

plt.figure(figsize=(14, 7))
sns.barplot(x='target', y='rmse', hue='experiment', data=comparison_df, palette='coolwarm',
            order=sorted(comparison_df['target'].unique(), key=lambda x: int(x.split('d')[0].split('_')[-1]))) # Сортировка таргетов

plt.title('Сравнение RMSE: Индивидуальное (среднее) vs Объединенное обучение', fontsize=16)
plt.xlabel('Таргет')
plt.ylabel('RMSE')
plt.xticks(rotation=45)
plt.legend(title='Тип эксперимента')
plt.grid(axis='y', linestyle='--')
plt.tight_layout()
plt.show()

# Отдельные графики по моделям для лучшей читаемости
for model_name in ['LightGBM', 'CatBoost']:
    plt.figure(figsize=(10, 5))
    model_data = comparison_df[comparison_df['model'] == model_name]
    sns.barplot(x='target', y='rmse', hue='experiment', data=model_data, palette='coolwarm',
                order=sorted(model_data['target'].unique(), key=lambda x: int(x.split('d')[0].split('_')[-1])))
    plt.title(f'Сравнение RMSE для модели: {model_name}', fontsize=14)
    plt.xlabel('Таргет')
    plt.ylabel('RMSE')
    plt.xticks(rotation=45)
    plt.legend(title='Тип эксперимента')
    plt.grid(axis='y', linestyle='--')
    plt.tight_layout()
    plt.show()

In [ ]:
# --- Визуализация: Сравнение Directional Accuracy ---

plt.figure(figsize=(14, 7))
sns.barplot(x='target', y='dir_acc', hue='experiment', data=comparison_df, palette='viridis',
            order=sorted(comparison_df['target'].unique(), key=lambda x: int(x.split('d')[0].split('_')[-1]))) # Сортировка таргетов

plt.title('Сравнение Точности Угадывания Знака (%): Индивидуальное (среднее) vs Объединенное', fontsize=16)
plt.xlabel('Таргет')
plt.ylabel('Точность угадывания знака (%)')
plt.xticks(rotation=45)
plt.legend(title='Тип эксперимента')
plt.grid(axis='y', linestyle='--')
plt.ylim(bottom=min(40, comparison_df['dir_acc'].min() - 2 )) # Установим нижнюю границу оси Y для наглядности
plt.tight_layout()
plt.show()

# Отдельные графики по моделям
for model_name in ['LightGBM', 'CatBoost']:
    plt.figure(figsize=(10, 5))
    model_data = comparison_df[comparison_df['model'] == model_name]
    sns.barplot(x='target', y='dir_acc', hue='experiment', data=model_data, palette='viridis',
                 order=sorted(model_data['target'].unique(), key=lambda x: int(x.split('d')[0].split('_')[-1])))
    plt.title(f'Сравнение Точности Угадывания Знака для модели: {model_name}', fontsize=14)
    plt.xlabel('Таргет')
    plt.ylabel('Точность угадывания знака (%)')
    plt.xticks(rotation=45)
    plt.legend(title='Тип эксперимента')
    plt.grid(axis='y', linestyle='--')
    plt.ylim(bottom=min(40, model_data['dir_acc'].min() - 2 ))
    plt.tight_layout()
    plt.show()

In [ ]:
# --- 1. Анализ корреляций ---

# Выберем тикер и таргет для анализа
analyze_ticker = 'SBER'
analyze_target = 'target_7d'
top_n_corr = 15 # Количество признаков для вывода

if analyze_ticker in split_data and analyze_target in split_data[analyze_ticker]:
    print(f"--- Анализ корреляций для {analyze_ticker}, таргет: {analyze_target} ---")

    # Используем оригинальные данные трейна (с импутацией, но до масштабирования)
    X_train_analyze = split_data[analyze_ticker][analyze_target]['X_train_orig']
    y_train_analyze = split_data[analyze_ticker][analyze_target]['y_train']

    # Объединим признаки и таргет для расчета корреляции
    train_data_analyze = pd.concat([X_train_analyze, y_train_analyze], axis=1)

    # Рассчитаем матрицу корреляций
    correlation_matrix = train_data_analyze.corr()

    # Корреляция всех признаков с целевой переменной
    corr_with_target = correlation_matrix[analyze_target].sort_values(ascending=False)

    print(f"\nТоп-{top_n_corr} признаков по абсолютной корреляции с {analyze_target}:")
    # Берем топ по модулю, исключая сам таргет (corr=1)
    top_correlated_features = corr_with_target.drop(analyze_target).abs().nlargest(top_n_corr)
    display(corr_with_target.loc[top_correlated_features.index])

    # Визуализация части матрицы корреляций (Топ N признаков + таргет)
    cols_to_plot = top_correlated_features.index.tolist() + [analyze_target]
    plt.figure(figsize=(12, 10))
    sns.heatmap(correlation_matrix.loc[cols_to_plot, cols_to_plot],
                annot=False, # Аннотации могут сделать график нечитаемым при большом N
                cmap='coolwarm',
                linewidths=0.5)
    plt.title(f'Матрица корреляций для топ-{top_n_corr} признаков и таргета ({analyze_ticker} / {analyze_target})')
    plt.show()

else:
    print(f"Не найдены данные для анализа корреляций: {analyze_ticker} / {analyze_target}")

In [ ]:
# --- 2. Важность признаков из моделей ---
top_n_features = 20 # Количество признаков для отображения на графиках
feature_importances = {} # Словарь для хранения важности

if analyze_ticker in split_data and analyze_target in split_data[analyze_ticker]:
    print(f"\n--- Анализ важности признаков из моделей для {analyze_ticker}, таргет: {analyze_target} ---")

    # Данные для обучения (оригинальные импутированные)
    X_train_fit = split_data[analyze_ticker][analyze_target]['X_train_orig']
    y_train_fit = split_data[analyze_ticker][analyze_target]['y_train']
    X_val_fit = split_data[analyze_ticker][analyze_target]['X_val_orig']
    y_val_fit = split_data[analyze_ticker][analyze_target]['y_val']

    # Модели для анализа важности
    models_for_importance = {
        "XGBoost": xgb.XGBRegressor(random_state=42, n_estimators=100, early_stopping_rounds=10),
        "LightGBM": lgb.LGBMRegressor(random_state=42, n_estimators=100, learning_rate=0.1),
        "CatBoost": cb.CatBoostRegressor(random_state=42, iterations=100, verbose=0, early_stopping_rounds=10)
    }

    fig, axes = plt.subplots(len(models_for_importance), 1, figsize=(12, 6 * len(models_for_importance)))
    if len(models_for_importance) == 1: axes = [axes] # Handle single model case
    fig.suptitle(f'Важность признаков ({analyze_ticker} / {analyze_target})', fontsize=16, y=1.0)


    for i, (model_name, model) in enumerate(models_for_importance.items()):
        print(f"  Обучение {model_name} для получения важности...")
        try:
            # Обучение (как в предыдущем цикле)
            if model_name == "XGBoost":
                model.fit(X_train_fit, y_train_fit, eval_set=[(X_val_fit, y_val_fit)], verbose=False)
                importances = model.feature_importances_
            elif model_name == "LightGBM":
                callbacks = [lgb.early_stopping(stopping_rounds=10, verbose=False)]
                model.fit(X_train_fit, y_train_fit, eval_set=[(X_val_fit, y_val_fit)], callbacks=callbacks)
                importances = model.feature_importances_
            elif model_name == "CatBoost":
                 model.fit(X_train_fit, y_train_fit, eval_set=[(X_val_fit, y_val_fit)], verbose=0)
                 importances = model.get_feature_importance()

            # Сохранение и визуализация
            imp_df = pd.DataFrame({'feature': X_train_fit.columns, 'importance': importances})
            imp_df = imp_df.sort_values(by='importance', ascending=False).head(top_n_features)
            feature_importances[model_name] = imp_df

            # График
            ax = axes[i]
            sns.barplot(x='importance', y='feature', data=imp_df, ax=ax, palette='rocket')
            ax.set_title(f'Топ-{top_n_features} признаков для {model_name}')
            ax.set_xlabel('Важность')
            ax.set_ylabel('Признак')

        except Exception as e:
            print(f"    Ошибка при получении важности для {model_name}: {e}")

    plt.tight_layout(rect=[0, 0, 1, 0.98])
    plt.show()

    # Сравним топ признаков между моделями
    if feature_importances:
        print("\nТоп-5 признаков по моделям:")
        comparison_list = []
        for model_name, imp_df in feature_importances.items():
             comparison_list.append(pd.Series(imp_df['feature'].values[:5], name=model_name))
        if comparison_list:
             comparison_df = pd.concat(comparison_list, axis=1)
             display(comparison_df)

else:
    print(f"Не найдены данные для анализа важности: {analyze_ticker} / {analyze_target}")

In [ ]:
selected_base_features_template = [
    'OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOLUME', 
    'RSI', 'MACD_Hist', 'Bear_Power', 'ATR', 'EMA_10', 'EMA_50', 'EMA_200', 'RTSI', 'EMV', 'ADX',
    'MOEXCN', 'MOEXIT', 'MOEXRE', 'MOEXEU', 'MOEXFN', 'MOEXINN', 'MOEXMM', 'MOEXOG', 'MOEXTL', 'MOEXTN', 'MOEXCH',
    'Revenue_y', 'NetProfit_y', 'ROE_y', 'Assets_q', 'NetProfit_q', 'PB_q',
    'BRENT_CLOSE', 'KEY_RATE', 'USD_RUB',
    'PE_y', 'PB_y'
]


    # 2. Определение новостных и блоговых признаков (только для тикера, без WeightedIndices)
    current_news_features_specific = [
        f'{ticker}_news_score', 
        f'{ticker}_news_score_roll_avg_5',
        f'{ticker}_news_score_roll_avg_30',
        'WeightedIndices_news_score'
    ]
    current_blog_features_specific = [
        f'{ticker}_blog_score',
        f'{ticker}_blog_score_roll_avg_5',
        f'{ticker}_blog_score_roll_avg_30',
        'WeightedIndices_blog_score'
    ]